In [1]:
# %pip install llama-index qdrant_client torch transformers

# %pip install llama-index-embeddings-huggingface

# %pip install llama-index-llms-ollama

# %pip install llama-index-vector-stores-qdrant

# %pip install llama-index-embeddings-fastembed

In [2]:
import nest_asyncio
import qdrant_client

from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore

from IPython.display import Markdown, display

In [3]:
# set up asyncio
nest_asyncio.apply()

###  Define the LLM, the embedding model and re-ranker

In [4]:
llm = Ollama(model="llama3.2:1b", request_timeout=120.0)

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-large-en-v1.5")

rerank = SentenceTransformerRerank(model="BAAI/bge-reranker-base", top_n=2)

/Users/rakeshk94/miniconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
Settings.embed_model = embed_model

Settings.llm = llm

### Read the documents

In [6]:
input_dir_path = './data'

loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".txt"],
            recursive=True
        )
docs = loader.load_data()

### Set up the Qdrant vector database

In [7]:
client = qdrant_client.QdrantClient(host="localhost", port=6333)

vector_store = QdrantVectorStore(client=client,
                                 collection_name="document_chat")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(docs,
                                        storage_context=storage_context)

### Define the query engine and prompt template

In [8]:
query_engine = index.as_query_engine(similarity_top_k=4,
                                     node_postprocessors=[rerank])

template = """Context information is below.
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner. Incase 
              you don't know the answer say 'I don't know!'.
              
              Query: {query_str}
              
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

### Query the document

In [9]:
response = query_engine.query("""How did the structure of funding startups 
                                 in batches contribute to the success and 
                                 growth of the Y Combinator program and the
                                 startups involved?""")

In [10]:
display(Markdown(str(response)))

The structure of funding startups in batches, as implemented by YC (then known as Y Combinator), contributed significantly to its success and growth. Here's a step-by-step breakdown:

1. **Convenience for YC**: By funding startups in batches, YC could support multiple startups simultaneously, allowing them to focus on their projects without being tied down to individual founders.

2. **Isolation reduction**: Funding startups in batches helped alleviate the isolation problem faced by founders. With colleagues from different batches working together, they shared knowledge and expertise, facilitating collaboration and reducing individual founder problems.

3. **Standardized support**: The batch funding structure allowed for a standardized approach to providing support to each startup. This consistency enabled YC to offer efficient and effective mentorship, helping startups grow rapidly.

4. **Increased scalability**: By supporting multiple startups at once, YC could scale its efforts more efficiently, allowing it to handle a larger number of projects simultaneously without sacrificing quality or attention to detail.

5. **Improved alumni network**: The batch funding model enabled the creation of an active and engaged alumni community within YC. Alumni became familiar with each other's challenges, provided valuable advice, and collaborated on opportunities that benefited both the founders and the organization as a whole.

6. **Enhanced startup ecosystem**: By supporting multiple startups in batches, YC fostered a vibrant startup ecosystem, where new companies could interact with successful ones, learn from their experiences, and gain access to resources and expertise. This ecosystem contributed to the growth of the startup community and YC's reputation as a hub for innovation.

7. **Y Combinator culture**: The batch funding model became an integral part of Y Combinator's (YC) culture, shaping its approach to entrepreneurship and investing in startups. It helped establish YC as a leader in the startup accelerator space and paved the way for future growth and success.

8. **Alumni succession planning**: By providing support for multiple batches, YC prepared its alumni for potential future roles, ensuring that successful founders would continue to contribute to the organization's growth and success.

In summary, the structure of funding startups in batches was instrumental in YC's success by allowing it to provide standardized support, scale efficiently, and create a vibrant startup ecosystem.

In [11]:
# %pip install --upgrade ragas langchain_core


In [33]:
# %pip install ragas==0.1.7
# %pip install langchain==0.1.16
# %pip install unstructured
# %pip install python-magic-bin

In [34]:
# %pip uninstall langchain langchain_core ragas -y
# %pip install ragas


In [35]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [36]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader("./data/paul_graham")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)

documents = loader.load_and_split(text_splitter)

In [37]:
documents[0].to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'document', 'Document'],
 'kwargs': {'metadata': {'source': 'data/paul_graham/how_to_do_great_things.txt'},
  'page_content': 'How to Do Great Work\n\nJuly 2023\n\nIf you collected lists of techniques for doing great work in a lot of different fields, what would the intersection look like? I decided to find out by making it.\n\nPartly my goal was to create a guide that could be used by someone working in any field. But I was also curious about the shape of the intersection. And one thing this exercise shows is that it does have a definite shape; it\'s not just a point labelled "work hard."\n\nThe following recipe assumes you\'re very ambitious.\n\nThe first step is to decide what to work on. The work you choose needs to have three qualities: it has to be something you have a natural aptitude for, that you have a deep interest in, and that offers scope to do great work.\n\nIn practice you don\'t have to worry much about th

In [38]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

generator_llm = Ollama(model="phi3:3.8b")
critic_llm = Ollama(model="llama3.2:1b")

ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text"
)

### Create Ragas' TestsetGenerator

In [ ]:
# %pip install --upgrade ragas langchain


In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [ ]:

generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=ollama_emb
)

distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}
testset = generator.generate_with_langchain_docs(documents,
                                                 test_size=10,
                                                 distributions=distribution,
                                                 raise_exceptions=False)

In [ ]:
test_df = testset.to_pandas().dropna()

### Local Implementation of evaluating the RAG pipeline

In [52]:
import pandas as pd
test_df = pd.read_csv("./data/Test Data Paul Graham.csv").dropna()

In [53]:
def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }

In [55]:
from datasets import Dataset
from tqdm.auto import tqdm

test_questions = test_df["question"].values

responses = [generate_response(query_engine,q) for q in tqdm(test_questions)]

dataset_dict = {
    "question": test_questions,
    "answer": [response["answer"] for response in responses],
    "contexts": [response["contexts"] for response in responses],
    "ground_truth": test_df["ground_truth"].values.tolist()
}
ragas_eval_dataset = Dataset.from_dict(dataset_dict)

  0%|          | 0/47 [00:00<?, ?it/s]

In [56]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    context_recall,
    context_precision,
)

In [ ]:
metrics = [faithfulness, answer_correctness,
           context_recall, context_precision]

critic_llm = Ollama(model="llama3.2:1b")

ollama_emb = OllamaEmbeddings(model="nomic-embed-text")

evaluation_result = evaluate(
    llm=critic_llm,
    embeddings=ollama_emb,
    dataset=ragas_eval_dataset,
    metrics=metrics
)

In [59]:
eval_scores_df = pd.DataFrame(evaluation_result.scores)

In [ ]:
evaluation_result

In [ ]:
# pip install --upgrade ragas langchain numpy
